Як змінювалась структура генерації електроенергії за роками?

In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
data = pd.read_excel("2014-2020.xlsx")[["Час/Дата", "AES", "TEC", "VDE", "TES", "GES", "GAES_GEN"]]
data.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN
0,24-31.12.2020,9235,2039,621,4942,385.0,0
1,23-31.12.2020,9221,2159,707,5549,470.0,0
2,22-31.12.2020,9249,2377,709,5906,1000.0,0
3,21-31.12.2020,9256,2499,702,6329,909.0,322
4,20-31.12.2020,9213,2521,702,6640,823.0,602


In [4]:
data['year'] = pd.to_datetime(data['Час/Дата'].replace(regex=r'^24-', value='0-'), format='%H-%d.%m.%Y')
data = data.drop(columns=["Час/Дата"])

In [5]:
yearly_sum = data.groupby(data.year.dt.year).sum().reset_index()

In [6]:
yearly_sum.head()

,year,AES,TEC,VDE,TES,GES,GAES_GEN
0,2014,88204418,14684731,1606386,68605877,7991074.0,839842
1,2015,87413763,12041074,1234936,50259819,4964478.0,1554857
2,2016,80762094,12841033,1093190,49879543,7113989.0,1621377
3,2017,85314258,12208062,1177032,44945303,8531952.0,1575099
4,2018,84351328,12261018,1863726,47747092,10072767.0,1564867


In [7]:
sources = ["AES", "TEC", "VDE", "TES", "GES", "GAES_GEN"]
new_sources = ["АЕС", "ТЕЦ", "ВДЕ", "ТЕС", "ГЕС", "ГАЕС"]
new_sources_full = ["Атомна електростанції", "Теплоелектроцентралі", "Відновлювана енергетика", "Теплоелектростанції", "Гідроелектростанції", "Гідроакумулюючі електростанції"]
data_by_source = []

for i, source in enumerate(sources):
    temp_data = yearly_sum[['year', source]].rename(columns={source: "production"})
    temp_data["source"] = new_sources_full[i]
    data_by_source.append(temp_data)
    
working_data = pd.concat(data_by_source, ignore_index=True)
print(working_data["source"].unique())

working_data.head()

['Атомна електростанції' 'Теплоелектроцентралі' 'Відновлювана енергетика'
 'Теплоелектростанції' 'Гідроелектростанції'
 'Гідроакумулюючі електростанції']


,year,production,source
0,2014,88204418.0,Атомна електростанції
1,2015,87413763.0,Атомна електростанції
2,2016,80762094.0,Атомна електростанції
3,2017,85314258.0,Атомна електростанції
4,2018,84351328.0,Атомна електростанції


In [8]:
working_data["order_of_source"] = 0 
for i, source in enumerate(new_sources_full):
    current_subset = working_data['source'] == source
    working_data['order_of_source'] += (current_subset).astype('int32') * working_data[current_subset]['production'].mean()
    print(source, working_data[current_subset]['production'].mean())

Атомна електростанції 83622144.14285715
Теплоелектроцентралі 12974901.857142856
Відновлювана енергетика 2791686.4285714286
Теплоелектростанції 49402749.28571428
Гідроелектростанції 7299299.142857143
Гідроакумулюючі електростанції 1434399.7142857143


In [10]:
working_data["production"] /= 10**6

In [11]:
alt.Chart(working_data, title={'text': "Генерація електроенергії в Україні"}).mark_bar().encode(
    x=alt.X('sum(production):Q', axis=alt.Axis(title='Вироблено електроенергії за рік в ТВт', titleFontSize=14)),
    y=alt.Y('year:N', axis=alt.Axis(title='Рік', titleFontSize=14)),
    color=alt.Color('source:N', scale = alt.Scale(scheme = 'set3')),
    order=alt.Order('order_of_source:N', sort="descending")
).properties(
    width = 800, height = 300, background = '#F9F9F9', padding = 10
).configure_title(fontSize=18).configure_legend(
    title=None,
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=5,
    cornerRadius=5
)

alt.Chart(...)